# Recommender Systems with Implicit Feedback

The objective of this lab is to build a recommender system using implicit feedback only (users expressing interest for some items). We use the [MovieLens](https://grouplens.org/datasets/movielens/) dataset, filtering user-movie pairs with positive rating (at least 4).

The recommender system is based on a neural network trained with the [triplet loss](https://en.wikipedia.org/wiki/Triplet_loss).

This lab is inspired by [that](https://github.com/m2dsupsdlclass/lectures-labs/blob/master/labs/03_neural_recsys/Implicit_Feedback_Recsys_with_the_triplet_loss_rendered.ipynb) proposed by Olivier Griesel and Charles Ollion.

## Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
import scipy.stats as stats
import pandas as pd

In [2]:
import os.path as op
from zipfile import ZipFile
from urllib.request import urlretrieve

In [3]:
from sklearn.metrics import roc_auc_score

In [4]:
import tensorflow as tf

In [5]:
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense
from keras.layers import Lambda, Dot
from keras.regularizers import l2
from keras.layers.merge import dot, concatenate
from keras.utils import plot_model

Using TensorFlow backend.


## Data loading

We use the [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/) dataset (1M ratings). It might be necessary to adapt the architecture of the neural nets and the hyperparameters used to train them to work on larger datasets like [MovieLens 10M](https://grouplens.org/datasets/movielens/10m/) or [Yahoo Songs](https://webscope.sandbox.yahoo.com/catalog.php?datatype=r).

In [6]:
dataset = "ml-1m"
filename = dataset + ".zip"
url = "http://files.grouplens.org/datasets/movielens/" + filename

if not op.exists(filename):
    print('Downloading %s to %s...' % (url, dataset))
    urlretrieve(url, filename)

if not op.exists(dataset):
    print('Extracting %s to %s...' % (filename, dataset))
    ZipFile(filename).extractall('.')

In [7]:
df_ratings = pd.read_csv(op.join(dataset, 'ratings.dat'), sep='::', 
                        names=["user_id", "item_id", "rating", "timestamp"], engine = 'python')

In [8]:
df_ratings.head()

,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [9]:
df_items = pd.read_csv(op.join(dataset, 'movies.dat'), sep='::',
                    names=['item_id', 'title', 'category'], engine = 'python')

In [10]:
# Some titles are missing!
df_items

,item_id,title,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [11]:
item_ids = np.array(df_items['item_id'].values)

In [12]:
titles = np.array(df_items['title'].values)

In [13]:
categories = np.array(df_items['category'].values)

## Data cleaning

We first represent data as a sparse matrix. 

In [14]:
# -1 to start indices at 0
row = np.array(df_ratings['user_id'].values) - 1
col = np.array(df_ratings['item_id'].values) - 1
data = df_ratings['rating'].values

In [15]:
rating_mat = sparse.csr_matrix((data, (row, col)))

In [16]:
rating_mat

<6040x3952 sparse matrix of type '<class 'numpy.int64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [17]:
# retain only movies with a title
rating_mat = rating_mat[:,item_ids - 1]

In [18]:
rating_mat

<6040x3883 sparse matrix of type '<class 'numpy.int64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [19]:
n_users, n_items = rating_mat.shape

In [20]:
n_items == len(titles)

True

In [21]:
# retain only movies with at least one rating
total_ratings = rating_mat.T.dot(np.ones(n_users))
index = np.argwhere(total_ratings).ravel()

In [22]:
len(index)

3706

In [23]:
rating_mat = rating_mat[:,index]

In [24]:
rating_mat

<6040x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 1000209 stored elements in Compressed Sparse Row format>

In [25]:
n_users, n_items = rating_mat.shape

In [26]:
titles = titles[index]

In [27]:
categories = categories[index]

In [28]:
n_items == len(titles)

True

## Some statistics

In [29]:
# Overall ratings 
stats.describe(rating_mat.data)

DescribeResult(nobs=1000209, minmax=(1, 5), mean=3.581564453029317, variance=1.2479165329363373, skewness=-0.5536090572523492, kurtosis=-0.3519750423968597)

In [30]:
# Views (binary matrix)
view_mat = rating_mat.copy()
view_mat.data = np.ones_like(view_mat.data)

In [31]:
user_views = view_mat.dot(np.ones(n_items))
stats.describe(user_views)

DescribeResult(nobs=6040, minmax=(20.0, 2314.0), mean=165.5975165562914, variance=37151.41721522576, skewness=2.743966112411747, kurtosis=11.1920071429534)

In [32]:
item_views = view_mat.T.dot(np.ones(n_users))
stats.describe(item_views)

DescribeResult(nobs=3706, minmax=(1.0, 3428.0), mean=269.88909875876953, variance=147492.74154374897, skewness=2.813796491583702, kurtosis=10.661899563165777)

In [33]:
# Ratings
user_ratings = rating_mat.dot(np.ones(n_items)) / user_views
stats.describe(user_ratings)

DescribeResult(nobs=6040, minmax=(1.0153846153846153, 4.962962962962963), mean=3.702704866999724, variance=0.18457513358365776, skewness=-0.5784169868011894, kurtosis=1.0670999843652567)

In [34]:
item_ratings = rating_mat.T.dot(np.ones(n_users)) /  item_views
stats.describe(item_ratings)

DescribeResult(nobs=3706, minmax=(1.0, 5.0), mean=3.2388921779108912, variance=0.45282771122020127, skewness=-0.6257786213568961, kurtosis=0.3246479366504329)

## Implicit feedback

We now filter positive ratings (>= 4) to get implicit feedback, as a binary matrix.

In [35]:
feedback = rating_mat >= 4

In [36]:
feedback

<6040x3706 sparse matrix of type '<class 'numpy.bool_'>'
	with 575281 stored elements in Compressed Sparse Row format>

In [37]:
# Filter only users with at least 20 ratings
user_feedback = feedback.dot(np.ones(n_items))
index = np.argwhere(user_feedback >= 20).ravel()

In [38]:
len(index)

5180

In [39]:
feedback = feedback[index]

In [40]:
feedback

<5180x3706 sparse matrix of type '<class 'numpy.bool_'>'
	with 562800 stored elements in Compressed Sparse Row format>

In [41]:
n_users, n_items = feedback.shape

In [42]:
# Remove items with no feedback
item_feedback = feedback.T.dot(np.ones(n_users))
index = np.argwhere(item_feedback).ravel()

In [43]:
len(index)

3526

In [44]:
feedback = feedback[:,index]

In [45]:
feedback

<5180x3526 sparse matrix of type '<class 'numpy.bool_'>'
	with 562800 stored elements in Compressed Sparse Row format>

In [46]:
titles = titles[index]

In [47]:
categories = categories[index]

In [48]:
n_users, n_items = feedback.shape

In [49]:
# User feedback
user_feedback = feedback.dot(np.ones(n_items))
stats.describe(user_feedback)

DescribeResult(nobs=5180, minmax=(20.0, 1435.0), mean=108.64864864864865, variance=11592.10861952897, skewness=2.862125739601998, kurtosis=14.48003819018539)

In [50]:
# Item feedback
item_feedback = feedback.T.dot(np.ones(n_users))
stats.describe(item_feedback)

DescribeResult(nobs=3526, minmax=(1.0, 2608.0), mean=159.61429381735678, variance=76854.96551477777, skewness=3.558113622862412, kurtosis=16.86943490652283)

## Split into train set / test set

We keep 10 items per user for the test set.

In [63]:
train_feedback = feedback.copy()

In [64]:
for i in range(n_users):
    index = np.random.choice(feedback[i].indices, size = 10, replace = False)
    train_feedback[i,index] = 0

In [65]:
test_feedback = feedback - train_feedback

In [66]:
test_feedback

<5180x3526 sparse matrix of type '<class 'numpy.bool_'>'
	with 51800 stored elements in Compressed Sparse Row format>

In [67]:
test_feedback.dot(np.ones(n_items))

array([10., 10., 10., ..., 10., 10., 10.])

In [68]:
train_feedback

<5180x3526 sparse matrix of type '<class 'numpy.bool_'>'
	with 562800 stored elements in Compressed Sparse Row format>

In [69]:
# remove null entries 
train_feedback = feedback - test_feedback

In [70]:
train_feedback

<5180x3526 sparse matrix of type '<class 'numpy.bool_'>'
	with 511000 stored elements in Compressed Sparse Row format>

In [71]:
# User feedback in train set
train_user_feedback = train_feedback.dot(np.ones(n_items))
stats.describe(train_user_feedback)

DescribeResult(nobs=5180, minmax=(10.0, 1425.0), mean=98.64864864864865, variance=11592.10861952897, skewness=2.862125739601998, kurtosis=14.48003819018539)

In [72]:
# Item feedback in train set
train_item_feedback = train_feedback.T.dot(np.ones(n_users))
stats.describe(train_item_feedback)

DescribeResult(nobs=3526, minmax=(0.0, 2218.0), mean=144.92342597844583, variance=61176.069595748704, skewness=3.4262272577891633, kurtosis=15.496698957234099)

## Triplet loss

We now build a neural network with the triplet loss. The similarity score between a user and an item is given by the cosine similarity of their respective embeddings. This score will be used to recommend items to any user.

Training is achieved by forcing the network to give a higher score to positive items (seen by the user) than to  negative items (not seen by the user). 

<img src="https://perso.telecom-paris.fr/bonald/images/triplet-loss.jpg" style="width: 600px;" />

In [89]:
# Loss

def identity_loss(y_true, y_pred):
    """Ignore y_true and return the mean of y_pred
    
    This is a hack to work-around the design of the Keras API that is
    not really suited to train networks with a triplet loss by default.
    """
    return tf.reduce_mean(y_pred + 0 * y_true)


def difference_loss(inputs, margin=.5):
    """Triplet loss.
    
    If the inputs are cosine similarities, they range in
    (-1, 1) and their difference is in (-2, 2), so a margin of 1 is suitable.

    If the input similarities are not normalized, it might be necessary to 
    tune the margin.
    """
    positive_pair_sim, negative_pair_sim = inputs
    return tf.maximum(negative_pair_sim - positive_pair_sim + margin, 0)

In [90]:
# Network

def build_models(n_users, n_items, latent_dim=64):
    """Build a triplet model and its companion, the match model
    
    The triplet model is used to train the weights of the 
    match model. The triplet model takes 1 user, 1 positive item, 
    1 negative item and is trained with the triplet loss.
    
    The match model takes 1 user and 1 item as input and returns
    the similarity score.
    """
    # inputs
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')

    # embeddings
    user_layer = Embedding(n_users, latent_dim, 
                           name='user_embedding')
    item_layer = Embedding(n_items, latent_dim, 
                           name="item_embedding")

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # similarity as cosine similarity between embeddings
    positive_similarity = Dot(name="positive_similarity",
                              axes=1, normalize=True)(
        [user_embedding, positive_item_embedding])
    negative_similarity = Dot(name="negative_similarity",
                              axes=1, normalize=True)(
        [user_embedding, negative_item_embedding])

    # triplet loss net, only used for training
    triplet_loss = Lambda(difference_loss,
                          name='triplet_loss',
                          output_shape=(1,))([positive_similarity, negative_similarity])

    triplet_model = Model(name='Triplet loss net',inputs=[user_input,
                                  positive_item_input,
                                  negative_item_input],
                          outputs=triplet_loss)
    
    # match model, used to rank items
    match_model = Model(name='Match net', inputs=[user_input, positive_item_input],
                        outputs=positive_similarity)
    
    return triplet_model, match_model

In [91]:
triplet_model, match_model = build_models(n_users, n_items)

In [92]:
print(triplet_model.summary())

Model: "Triplet loss net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
positive_item_input (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
negative_item_input (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 64)        331520      user_input[0][0]                 
___________________________________________________________________________________

In [93]:
print(match_model.summary())

Model: "Match net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
positive_item_input (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 64)        331520      user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 64)        225664      positive_item_input[0][0]        
__________________________________________________________________________________________

## Metrics

We need a metric to assess the quality of the recommander system. Specifically, we would like to check that, for each user, items in the test set tend to get high scores compared to other unseen items (not in the train set). 
We use the [ROC AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) metric:
* ROC = Receiver Operating Characteristic = True positive rate (fraction of the test set above the threshold) against False positive rate (fraction not in the test set above the threshold) for various thresholds on the score. 
* AUC = Area Under Curve

Observe that the ROC AUC is between 0 and 1. It can be interpreted as the probability that a random pair of items, one in the test set (positive feedback) and the other not in the test set (no feedback) is correctly ranked by the model, for any given user. There is one ROC AUC metric per user.

In [ ]:
def model_scores(match_model):
    """Compute the ROC AUC score of each user"""
    scores = []
    for i in range(n_users):
        # items not in the training set
        items_to_rank = np.setdiff1d(np.arange(n_items), train_feedback[i].indices)
        
        # items in the test set = 1, others = 0
        expected = np.in1d(items_to_rank, test_feedback[i].indices)
        
        # model prediction
        repeated_user = np.repeat(i, repeats=len(items_to_rank))
        predicted = match_model.predict([repeated_user, items_to_rank], batch_size=4096).ravel()
        
        # ROC AUC
        scores.append(roc_auc_score(expected, predicted))

    return scores

## To do

* What is the expected ROC AUC of a random score function?
* Check your guess on the current model (not yet trained).

## Network training

Let's now train the network by sampling triplets.

## To do

* Complete the following function that sample triplets.
* Train the network and check the ROC AUC.
* List the 20 items recommanded by the model (unseen by the user) for the user with the highest and the user with the lowest ROC AUC. <br> How many of these items are in the test set?

In [ ]:
def sample_triplets():
    """Sample negatives at random among items not in the train set"""
    
    users = []
    pos_items = []
    neg_items = []
    
    for i in range(n_users):
        items = train_feedback[i].indices
        users += list(i * np.ones_like(items))
        pos_items += list(items)
        # to be modified
        neg_items += list(np.zeros_like(items))

    return [np.array(users), np.array(pos_items), np.array(neg_items)]

In [ ]:
triplet_model, match_model = build_models(n_users, n_items)

In [ ]:
# we plug the identity loss and a fake target variable ignored by
# the model to be able to use the Keras API

triplet_model.compile(loss=identity_loss, optimizer="adam")
fake_y = np.ones_like(train_feedback.data)

n_epochs = 3

for i in range(n_epochs):
    triplets = sample_triplets()
    triplet_model.fit(triplets, fake_y, shuffle=True, batch_size=64, epochs=1)
    

## Baseline

We would like to assess the quality of the recommendation compared to a simple baseline using the popularity of the items (= number of positive feedback in the train set). Observe that the recommendation is no longer personalized.

## To do

* Compare the baseline to the model in terms of ROC AUC.
* Compare the 20 first unseen items recommended by the model and by the baseline. You might estimate the number of common items averaged over all  users.

## To go further

* Propose and test a method to get similar items (e.g., the 10 movies the most similar to Fargo).
* Propose and test a method to include metadata in the model.
* Propose and test a recommender system using graph algorithms (e.g., PageRank).